In [1]:
# Mengunduh file ZIP dari repositori
!wget https://github.com/vis-nlp/Chart-to-text/archive/refs/heads/main.zip

--2025-07-01 05:55:30--  https://github.com/vis-nlp/Chart-to-text/archive/refs/heads/main.zip
Resolving github.com (github.com)... 4.237.22.38
Connecting to github.com (github.com)|4.237.22.38|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/vis-nlp/Chart-to-text/zip/refs/heads/main [following]
--2025-07-01 05:55:31--  https://codeload.github.com/vis-nlp/Chart-to-text/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 4.237.22.35
Connecting to codeload.github.com (codeload.github.com)|4.237.22.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [              <=>   ]   1.73G  20.8MB/s    in 95s     

2025-07-01 05:57:06 (18.7 MB/s) - ‘main.zip’ saved [1860014607]



In [2]:
# Mengekstrak isi file ZIP
!unzip -q main.zip

In [3]:
import pandas as pd
metadata_path = "/kaggle/working/Chart-to-text-main/statista_dataset/dataset/metadata.csv"
metadata = pd.read_csv(metadata_path)
metadata

,id,title,dataPath,imgPath,caption,first_caption,chartType,release date,Region,survey time period,xAxis,yAxis,URL
0,1,\n Number of monthly ac...,out/two_col/data/1.csv,out/two_col/imgs/1.png,How many users does Facebook have? With over 2...,How many users does Facebook have? With over 2...,line,Oct-20,Worldwide,Q3 2008 to Q3 2020,Unnamed: 0,Number of users in millions,https://www.statista.com/statistics/264810/num...
1,2,\n United States: estim...,out/two_col/data/2.csv,out/two_col/imgs/2.png,This statistic presents the estimated net wor...,This statistic presents the estimated net wor...,bar,Mar-20,United States,Mar-20,Unnamed: 0,Net worth in billion U.S. dollars,https://www.statista.com/statistics/201426/the...
2,3,\n Reported violent cri...,out/two_col/data/3.csv,out/two_col/imgs/3.png,This statistic shows the reported violent cri...,This statistic shows the reported violent cri...,line,Sep-20,United States,1990 to 2019,Unnamed: 0,"Reported violent crime rate per 100,000 popula...",https://www.statista.com/statistics/191219/rep...
3,4,\n Players with the mos...,out/two_col/data/4.csv,out/two_col/imgs/4.png,Emmitt Smith is the all-time rushing leader o...,Emmitt Smith is the all-time rushing leader o...,bar,Jan-21,United States,"as of January 4, 2021",Unnamed: 0,Number of yards,https://www.statista.com/statistics/784449/pla...
4,5,\n Players with most go...,out/two_col/data/5.csv,out/two_col/imgs/5.png,"As of July 2020, Cristiano Ronaldo leads the ...","As of July 2020, Cristiano Ronaldo leads the ...",bar,Mar-20,Europe,1955 to 2020,Unnamed: 0,Goals scored,https://www.statista.com/statistics/378059/cha...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27863,27864,\n Distribution of cybe...,out/two_col/data/27864.csv,out/two_col/imgs/27864.png,This statistic presents the distribution of c...,This statistic presents the distribution of c...,column,Feb-14,"Worldwide, North America",2013,Unnamed: 0,Share of respondents,https://www.statista.com/statistics/291220/cyb...
27864,27865,\n Total number of dwel...,out/two_col/data/27865.csv,out/two_col/imgs/27865.png,This statistic displays the total number of d...,This statistic displays the total number of d...,column,May-20,United Kingdom (Great Britain),2003 to 2017,Unnamed: 0,Number of dwellings in thousands,https://www.statista.com/statistics/291805/uk-...
27865,27866,\n Results in the Europ...,out/two_col/data/27866.csv,out/two_col/imgs/27866.png,This statistic shows the political parties an...,This statistic shows the political parties an...,bar,Jul-14,"United Kingdom, EU",2014,Unnamed: 0,Percentage of votes,https://www.statista.com/statistics/300468/eu-...
27866,27867,\n Average annual expen...,out/two_col/data/27867.csv,out/two_col/imgs/27867.png,This statistic shows the average annual expen...,This statistic shows the average annual expen...,column,Sep-20,United States,2007 to 2020,Unnamed: 0,Average annual expenditure in U.S. dollars,https://www.statista.com/statistics/305551/us-...


In [4]:
metadata.groupby('chartType').size()

chartType
bar        8272
column    16319
line       2646
pie         408
table       223
dtype: int64

In [5]:
import os
import shutil
import pandas as pd
import traceback

# Path ke dataset asli dan CSV hasil prediksi
original_dataset_path = "/kaggle/working/Chart-to-text-main/statista_dataset/dataset"
new_dataset_path = "/kaggle/working/new-dataset"

try:
    # Baca CSV hasil prediksi
    metadata = pd.read_csv(metadata_path)

    # Filter hanya untuk kelas 'barchart' dan 'piechart'
    filtered = metadata[metadata['chartType'].isin(['bar', 'column', 'pie'])]

    # Pastikan folder baru dibuat sesuai struktur
    folders_to_copy = {
        'imgs': '.png',
        'captions': '.txt',
        'data': '.csv',
        'titles': '.txt'
    }

    for folder in folders_to_copy.keys():
        os.makedirs(os.path.join(new_dataset_path, folder), exist_ok=True)

    # Copy file yang sesuai prediksi ke dataset baru
    for _, row in filtered.iterrows():
        image_id = row['id']  # ID gambar, sesuaikan dengan nama kolom di CSV
        for folder, file_ext in folders_to_copy.items():
            try:
                # Path ke file asli dan tujuan
                src_file = os.path.join(original_dataset_path, folder, f"{image_id}{file_ext}")
                dst_file = os.path.join(new_dataset_path, folder, f"{image_id}{file_ext}")

                # Copy file jika ada
                if os.path.exists(src_file):
                    shutil.copy(src_file, dst_file)
                else:
                    print(f"Warning: File {src_file} tidak ditemukan.")
            except Exception as e:
                print(f"Error saat memproses file di folder {folder} untuk ID {image_id}:")
                traceback.print_exc()

    print("Dataset baru berhasil dibuat!")

except Exception as e:
    print("Error utama terjadi:")
    traceback.print_exc()

Dataset baru berhasil dibuat!


In [6]:
import os

# Hitung jumlah file dalam setiap folder
for folder in folders_to_copy.keys():
    folder_path = os.path.join(new_dataset_path, folder)
    num_files = len(os.listdir(folder_path))
    print(f"Jumlah file dalam folder '{folder}': {num_files}")

# Hitung total file yang berhasil dicopy
total_files = sum(len(os.listdir(os.path.join(new_dataset_path, folder))) for folder in folders_to_copy.keys())
print(f"Total semua file yang berhasil dicopy: {total_files}")

Jumlah file dalam folder 'imgs': 24999
Jumlah file dalam folder 'captions': 24999
Jumlah file dalam folder 'data': 24999
Jumlah file dalam folder 'titles': 24999
Total semua file yang berhasil dicopy: 99996


In [7]:
folder_path = '/kaggle/working/new-dataset'
output_zip = 'new-dataset.zip'

shutil.make_archive(output_zip.replace('.zip', ''), 'zip', folder_path)

print(f"{output_zip} berhasil dibuat.")

new-dataset.zip berhasil dibuat.
